# Workshop 3
Hi and welcome to the third workshop. This session will focus on boundary conditions, specifically list type boundary conditions. Some boundary conditions also allow for specifying arrays but we will cover that in the next session along with other array style input packages. You will also be introduced to use and configuration of time series files plus use and configuration of observation files. Additionally, we will introduce specifying auxiliary variables as either multipliers or for transport modelling.  

As before please make sure you have the MF6io.pdf document open while working through this notebook because it will show you the model input file structure for the specific boundary/package that you are creating and provides a more complete reference of the different options availabl.
.

# Imports first

In [ ]:
import os
import sys
import shutil
import platform
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import flopy

print(f"Pandas version = {pd.__version__}")
print(f"Numpy version = {np.__version__}")
print(f"Flopy version = {flopy.__version__}")
print(f"Matplotlib version = {matplotlib.__version__}")

# Build a simple steady-state structured model

In [ ]:
# Setup some folders
ws3 = os.path.join('workshop_3') # here we are making a path not creating the folder
gis_f = os.path.join(ws3,'GIS') # creating a sub-directory path for our gis input/output
model_f = os.path.join(ws3,'model') # creating a sub-directory path for our model input/output
plots_f = os.path.join(ws3,'plots') # creating a sub-directory path for our plots
for path in [ws3,gis_f,model_f,plots_f]:
    if os.path.exists(path): # here we are asking if the path exists on the computer. 
        shutil.rmtree(path)# if it does exist, delete it and all the files in it
        os.mkdir(path) # then remake it
    else:
        os.mkdir(path) # if it doesn't exist then make the folder

#Build some basic model objects
simulation_name = 'MySim' # setting a name for our simulation
sim = flopy.mf6.MFSimulation(sim_name=simulation_name, sim_ws=model_f, exe_name='mf6') # for a no frills mfsim.nam file
tdis = flopy.mf6.ModflowTdis(sim) # for the MySim.tdis file, recall this sets a single steady-state stress period
ims = flopy.mf6.ModflowIms(sim) # for the MySim.ims file, recall this sets up IMS as 'SIMPLE'
model_name = 'flow' # setting a name for our model
gwf = flopy.mf6.ModflowGwf(sim, modelname=model_name, save_flows=True) # for the flow.nam file

# Recall that there are default values applied to whatever you don't specify. 
# What esle should have been specified? Check your mf6io.pdf document.
# Make sure you are aware of the default entries that Flopy uses when you don't specify something for a package.
# REMEBER
# *** as the modeller it is your responsibility to check your input files ***
nrow = 20
ncol = 20
dis = flopy.mf6.ModflowGwfdis(gwf, nrow=nrow, ncol=ncol) # why not just specify the values in here directly? Why use ncol=ncol?

# Taking a look at things

In [ ]:
# Lets get a modelgrid object and have a look at the grid
mg = gwf.modelgrid

# create a figure to save to our plots folder
fig,ax = plt.subplots(figsize=(5,5)) # we are creating a figure object here so that we can dictate size
mg.plot(ax=ax)
ax.ticklabel_format(style='plain') # test what happens if you don't use this switch
ax.set_title('DIS Model Grid')
ax.set_xlabel('Eastings')
ax.set_ylabel('Northings')
# lets save it to our plots folder
figname = os.path.join(plots_f,'DIS_model_grid.png') # note use of the plots folder path.
# If you want to change the file format then change the extension from .png to pdf or just do both
fig.savefig(figname,dpi=300)
figname = os.path.join(plots_f,'DIS_model_grid.pdf')
fig.savefig(figname,dpi=300) 

# While we are here we can export it as a shapefile too.
dis.export(os.path.join(gis_f,'dis.shp'))
# ignore the warnings

# The stress period data dictionary
This is arguably the most important part of setting up boundary conditions. Once you grasp how to use and build the information needed in the stress period dictionary, the setup of a basic boundary conditions becomes reasonably straight forward. Whatever variables you see in the MF6io.pdf document between BEGIN PERIOD X and END PERIOD X will have to included in the dictionary. A dictionary is not always necessary as will be demonstrated but learning how to use them will be of benefit when it comes to setting up more complex versions of the same boundary. Boundary conditions are specific to a model so the model object must exist and be provided. Check the options block of whatever boundary condition you want to setup in the MF6io.pdf. Most of the settings in the options block are set with simple entries like option = "True", option= None, option = string, option = list_of_strings or option = integer etc. Check the Flopy defaults for each option and make sure you understand what it means to turn different settings on or off.

# Active or Inactive
You need to pay special attention to how the specific boundary you are configuring is made active or inactive. This varies for some boundaries and is noted in the MF6io.pdf document, so please be sure to read the specifics for the boundary you are working on. For the boundaries we will demonstrate in this workshop they are active from the first stress period you designate and will remain active until explicitly deactivated in a subsequent stress period. If you don't designate a stress period in which a boundary condition should first activate then Flopy assumes you want the boundary active throughout the entire simulation. You should always check your package input file before running your model anyway to ensure that everything is correct.

# Assign a constant head
To begin with we will assign a first type boundary because it is relatively simple and then we will demonstrate all the different boundary condition features offered in MF6.

In [ ]:
# Assigning a constant head bounadry to specific model cells to be active all the time.

chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=[[(0, 0, 0), 10.],
                                                       [(0, 19, 19), 8.]])

# which model cells were assigned? what value were they given?
# recall we have a structured model so what do the numbers in the tuples represent?
# If this were a DISV grid, how would this input change?
# What if it was DISU?

# Writing your boundary condition package
If you want to be able to write your input file on its own then you need to name the object. What I am referring to here is the "chd = " part. You can instantiate the object without doing this and it will still work. We have been assigning names to the objects all along because it allows us to easily access the built in methods for that object. You've seen how we write a simulation before in Workshop 1. Writing the simulation will write all the model files you've created so far but here we only want the CHD package file we just created.

In [ ]:
chd.write()
# note that it will determine what value needs to be assigned to the maxbound variable 
# based on the information you provide in your stress_period_data.

# Look for the file in your model folder and open it using your referred text editor or use the box below

In [ ]:
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,f"{model_name}.chd"))]
# note the name of the input file will be the model name followed by the package Ftype acronym. 

# Plotting the boundary to check it
It is always a good idea to visually check that the cells you intended to set the boundary on are correct. You can do this by plotting the model grid and also plotting a specific boundary condition.

In [ ]:
# Use the same method we applied previously in Workshop 2 to plot the modelgrid using a MapView object
# remeber the sequence of steps
# 1. Get modelgrid
# 2. Create figure instance
# 3. Create Mapview object
# 4. Tell the Mapview object what you want to plot
# 5. Modify your figure and axes to your preferred settings
# Note the line used to plot the bounadry condition

# Lets get a modelgrid object and have a look at the grid
mg = gwf.modelgrid

# create a figure instance to save to our plots folder
fig,ax = plt.subplots(figsize=(7,5)) # we are creating a figure object here so that we can dictate size note there are mutiple ways to do this

# Create the mapview object of the modelgrid and tell it to use the axes we created with our figure
pmv = flopy.plot.PlotMapView(modelgrid=mg, ax=ax) # note by not specifying a layer here it will assume layer 1

# say that you want the Mapview object to plot the grid with linewidth = 0.3 and colour the lines black
pmv.plot_grid(ax=ax, lw=0.3, color="black")

# Now I also want to plot the boundary condition so I specify the package object to plot
pmv.plot_bc(package=chd)


ax.set_title('CHD Boundary Cells')
ax.set_xlabel('Eastings')
ax.set_ylabel('Northings')
# lets save it to our plots folder
figname = os.path.join(plots_f,'CHD_bounadry_cells.png') # note use of the plots folder path.
# If you want to change the file format then change the extension from .png to pdf or just do both
fig.savefig(figname,dpi=300)
figname = os.path.join(plots_f,'CHD_bounadry_cells.pdf')
fig.savefig(figname,dpi=300) 

There are other options available when plotting a boundary condition that we will explore later. This includes specifying the layer you wish to view if you have a multi-layer model, specifying a stress period if you have boundary conditions that vary between active and inactive during your simulation, changing the colour of the plotted boundary and if you want to plot all boundaries.

# Selecting cells for a boundary condition
The first example demonstrates how to select individual cells in a structured grid but we may also want to assign a boundary condition along the edge of a model grid. Selecting model cells to assign a boundary condition to is half the battle so having different methods for selecting cells is going o help you a lot. We'll stick with our structured grid for now and look at selecting cells using some list comprehensions. We are going to assign our boundaries down the left and right side of our model as opposed to the corners. This means we need to build our stress period dictionary a little bit differently


In [ ]:
# lets first get rid of our existing chd object
del chd 

# We only have one layer so if using l,r,c it means that we always have l=1 (or 0 for zero based)
l=0
# If we are on the left boundary then c is always 1 (or 0 for zero based) 
c = 0
# this means that the only variable we need to loop over is r
left_bound_nodes = [(l,r,c) for r in range(nrow)]
# lets take a look at it
left_bound_nodes

So that sorts out the nodes but the stress period dictionary needs more information than that. We also need to specify the value of the constant head in those cells. Each entry in the period block of the input file needs to provided to Flopy as a list with the node id first then the rest of the boundary information, which is the constant head value in this case.

In [ ]:
# Now we just add on the rest of the information that we want to include in the list comprehension and make each loop provide a list.
left_bound_stress_period_block = [[(l,r,c),10.] for r in range(nrow)]
left_bound_stress_period_block
# note what you produced is a list of lists evidenced by the leading and trailing double brackets

In [ ]:
# but we also need it for the right boundary so lets do the same thing here
# c on the right boundary is the number of columns minus one for zero base
c = ncol-1
right_bound_stress_period_block = [[(l,r,c),8.] for r in range(nrow)]
right_bound_stress_period_block

In [ ]:
# We need our left and right boundaries incorporated into a single list for the stress period dictionary so lets unpack them
chd_period_list = [*right_bound_stress_period_block,*left_bound_stress_period_block]
chd_period_list

In [ ]:
# Now we can build our new CHD bounadry but this time we provide our list as an object name
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_period_list)
# write it
chd.write()
# Look at the file
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,f"{model_name}.chd"))]

In [ ]:
# Then plot it to have a better look

# Lets get a modelgrid object and have a look at the grid
mg = gwf.modelgrid

# create a figure instance to save to our plots folder
fig,ax = plt.subplots(figsize=(7,5)) # we are creating a figure object here so that we can dictate size note there are mutiple ways to do this

# Create the mapview object of the modelgrid and tell it to use the axes we created with our figure
pmv = flopy.plot.PlotMapView(modelgrid=mg, ax=ax) # note by not specifying a layer here it will assume layer 1

# say that you want the Mapview object to plot the grid with linewidth = 0.3 and colour the lines black
pmv.plot_grid(ax=ax, lw=0.3, color="black")

# Now I also want to plot the boundary condition so I specify the package object to plot
pmv.plot_bc(package=chd)


ax.set_title('CHD Boundary Cells')
ax.set_xlabel('Eastings')
ax.set_ylabel('Northings')
# lets save it to our plots folder
figname = os.path.join(plots_f,'CHD_bounadry_cells_LR.png') # note use of the plots folder path.
# If you want to change the file format then change the extension from .png to pdf or just do both
fig.savefig(figname,dpi=300)
figname = os.path.join(plots_f,'CHD_bounadry_cells_LR.pdf')
fig.savefig(figname,dpi=300) 

# But wait there is a problem here
Take a look at the model object. There are two boundary packages assigned to the model object even though we deleted the first chd object and used the same object name. MF6 allows multiples of the same boundary type to be used. Previous versions of model only allowed one instance of each boundary package. Look at the default naming convention used by Flopy when you have multiple instances of the same package. You can overrule this through the options block, which we will do shortly but first we need to fix our extra chd package option.


In [ ]:
gwf

In [ ]:
print(gwf.get_package_list())
gwf.remove_package("CHD_0")
print(gwf.get_package_list())
# but which one did we keep?

But which one did we keep? Check by plotting to be sure.

In [ ]:
# Lets get a modelgrid object and have a look at the grid
mg = gwf.modelgrid

# create a figure instance to save to our plots folder
fig,ax = plt.subplots(figsize=(7,5)) # we are creating a figure object here so that we can dictate size note there are mutiple ways to do this

# Create the mapview object of the modelgrid and tell it to use the axes we created with our figure
pmv = flopy.plot.PlotMapView(modelgrid=mg, ax=ax) # note by not specifying a layer here it will assume layer 1

# say that you want the Mapview object to plot the grid with linewidth = 0.3 and colour the lines black
pmv.plot_grid(ax=ax, lw=0.3, color="black")

# Now I also want to plot the boundary condition so I specify the package object to plot
pmv.plot_bc(package=chd)


ax.set_title('CHD Boundary Cells')
ax.set_xlabel('Eastings')
ax.set_ylabel('Northings')
# lets save it to our plots folder
figname = os.path.join(plots_f,'CHD_bounadry_cells_LR.png') # note use of the plots folder path.
# If you want to change the file format then change the extension from .png to pdf or just do both
fig.savefig(figname,dpi=300)
figname = os.path.join(plots_f,'CHD_bounadry_cells_LR.pdf')
fig.savefig(figname,dpi=300) 

Lets delete the old input files and re-write our package

In [ ]:
_ = [os.remove(os.path.join(model_f,x)) for x in os.listdir(model_f)]
# for x in os.listdir(model_f) means give me all the names of the files in the model folder
# os.remove(os.path.join(model_f,x)) means remove the file at the path given by os.path.join(model_f,x) 

print(os.listdir(model_f)) # should be an empty list

In [ ]:
# now write our model file
chd.write()

# but which file name will it use now?
print(os.listdir(model_f))

In [ ]:
# Look at the file
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,'flow_0.chd'))]

# Run a model
The following import will provide a function for you to use so that you can run a model using what you have setup so far. This is similar to the introductory model on the Flopy GitHub site. It will also do some post processing for you so that we can view results quickly and focus on the boundary conditions.


In [ ]:
from helpers import ws3_mod1

In [ ]:
# Clear out all existing modle files
_ = [os.remove(os.path.join(model_f,x)) for x in os.listdir(model_f)]

In [ ]:
# run the function by passing in your sim and gwf objects
ws3_mod1(sim,gwf)

Let's try adding a well rouglhy at the centre of our model that extracts 1L/s (864 m3/d)

In [ ]:
# Take a look at the MF6io.pdf document to see what is needed in the well stress period data block
# then follow the same procedure we did previously
wel1 = [(0,9,9), -10.0]
wel_period_list = [wel1] # remember that the stress_period data is a list of lists
wel = flopy.mf6.ModflowGwfwel(gwf, stress_period_data=wel_period_list)

In [ ]:
# check our model object to make sure the well package is there
gwf

In [ ]:
# Clear out all existing modle files
_ = [os.remove(os.path.join(model_f,x)) for x in os.listdir(model_f)]

In [ ]:
# Run our model again
ws3_mod1(sim,gwf)

# What if I want to know the fluxes associated with the boundaries?
You can get this from the list file but this workshop is on boundary conditions so we'll get straight from there. First, we will have to assign our boundaries some names and then setup an observation package to go with each of them. Now we will start to use some of the options in the options block. Again take a look at the MF6io.pdf document to check what you can do with each boundary condition. We will start by removing our CHD and WEL packages.


In [ ]:
print(gwf.get_package_list())
gwf.remove_package("CHD") 
gwf.remove_package("WEL")
print(gwf.get_package_list())

# Why don't I have to remove the other packages as well?
The remaining packages will be overwritten because you can only have one of them assigned to a model. You can have several boundary conditions of the same type in each model so you need to delete the ones that you don't want or you may accidentally double up. Again it is your responsibility to check your input files before running your model.


# Adding boundary names

In [ ]:
# now our stress period data list will have to include some more information so we need to rebuild it
c = 0
# this is what we had before [[(l,r,c),10.]
left_bound_stress_period_block = [[(l,r,c),10.0,"left_chd"] for r in range(nrow)] # All we did was add in a string for this bounadry name
# now the right one
c = ncol-1
right_bound_stress_period_block = [[(l,r,c),8.0,"right_chd"] for r in range(nrow)]
chd_period_list = [*left_bound_stress_period_block,*right_bound_stress_period_block]
chd_period_list

# Setting up a boundary observation package

In [ ]:
# build obs data dictionary
chd_obs = {
    "chd_flux.csv": [                    # This is the user specified name of the file that the boundary fluxes will be stored in after the model has run
        ("Flux_left", "CHD", "left_chd"), # A tuple with what will be the column heading in the output csv file, the code for the type of observation , then the boundary name
        ("Flux_right", "CHD", "right_chd"), # A tuple with what will be the column heading in the output csv file, the code for the type of observation , then the boundary name
    ],
}
# So what you have is a dictionary with the output filename as the key
# note how the boundary name is used to efectivley group a bunch of boundary cells together
# We could have broken the boundary up into more sections with more names if wanted
# The code informing the observation package what it needs to observe differs for each package
# We will look through these in the workshop
chd_obs

In [ ]:
# Clear out all existing modle files
_ = [os.remove(os.path.join(model_f,x)) for x in os.listdir(model_f)]

In [ ]:
chd = flopy.mf6.ModflowGwfchd(gwf, 
                              boundnames=True, # Tell it we will be using boundary names
                              observations=chd_obs, # Tell it we will be using an observation package
                              stress_period_data=chd_period_list)
# write the boundary and its observation package
chd.write()
chd.obs.write()

In [ ]:
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,'flow.chd'))]
# Note the options have been added in

In [ ]:
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,'flow.chd.obs'))]
# this is what the observation file looks like
# check that it makes sense according to the mf6io.pdf document

In [ ]:
# Now we will do it for the well but in a slightly different way
wel1 = [(0,9,9), -10.0,"Mywell"] # adding in a boundary name
wel_period_list = [wel1] # remember that the stress_period data is a list of lists
wel = flopy.mf6.ModflowGwfwel(gwf, boundnames=True, stress_period_data=wel_period_list) 
# added the bound and built the object but didn't setup an observation package
# Lets do it now using an initialization method

In [ ]:
# setup the observation dictionary the same as before
wel_obs = {
    "wel_flux.csv": [
        ("Well rate", "WEL", "Mywell")]
}
wel.obs.initialize(continuous=wel_obs)

In [ ]:
# write the boundary and its observation package
wel.write()
wel.obs.write()

In [ ]:
# lets run our model to see what happens
ws3_mod1(sim,gwf)

# Lets try a transient model

In [ ]:
# we'll remove the existing tdis package from sim object
sim.remove_package("TDIS")

In [ ]:
del tdis # delete the existing tdis object
# now we build a new one
tdis = flopy.mf6.ModflowTdis(sim,
                             time_units='days',
                             start_date_time="2023-12-31T00:00:00",
                             nper=2,
                             perioddata=[(1,1,1),(365.25,52,1)]) # steady-state initial stress period
tdis

In [ ]:
# import a transient version of the same model we used before
from helpers import ws3_mod1_trans

# Changing to a transient stress period data dictionary
So far we have been using a list of lists to create our stress period data and this is fine for boundary conditions that don't vary throughout your simulation. But for this transient simulation we don't want our well active in the steady-state stress period. We want it to start in the second stress period.


In [ ]:
# remove the existing wel package from our simulation
gwf.remove_package("WEL")
# Now we build a new one starting with the stress period data DICTIONARY
# We still use the list of lists but now it is stored inside a dictionary 
# with the key representing the stress period number

wel_period_dict = {} # start an empty dictionary
wel1 = [(0,9,9), -10.0,"Mywell"] # adding in a boundary name
wel_period_list = [wel1] # remember that the stress_period data is a list of lists
wel_period_dict[1] = wel_period_list # note the dictioanry key is 1, this is stress period two (zero base)

wel = flopy.mf6.ModflowGwfwel(gwf, boundnames=True, stress_period_data=wel_period_dict) # Note we changed this to the dictionary 
# added the bound and built the object but didn't setup an observation package
# Lets do it now using an initialization method

# setup the observation dictionary the same as before
wel_obs = {
    "wel_flux.csv": [
        ("Well rate", "WEL", "Mywell")]
}
wel.obs.initialize(continuous=wel_obs)
# check the model object to make sure we haven't doubled up on boundaries
gwf.get_package_list()

In [ ]:
ws3_mod1_trans(sim,gwf)

In [ ]:
# Lets look at the well file
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,'flow.wel'))]

# Check the observation files now
Notice that they now have a value for each time step recorded. Not exactly thrilling stuff but lets change our model to vary the pumping and then maybe this will be more interesting to look at.


# What if I want to pump more in summer when there is less surface water and less in winter?
There are different ways to approach this we will do it with a time series because we can vary our stresses at the time step level so no need for lots of stress periods. Previous versions of MODFLOW required that you needed a stress period in order to change any values associated with any transient boundary conditions. The time series needs to extend beyond the activity of the boundary activation that depends on it. For example, if I have a boundary condition active for one year then ideally my time series should extend a day before and a day after to be safe. We will go through the options different options associated with time series files in the workshop or you can read more about how they are configured and what the different variables mean in the MF6io.pdf document. We need to rebuild the well package to change the assigned rates to a time series variable name.


In [ ]:
# remove the existing wel package from our simulation
gwf.remove_package("WEL")
# Now we build a new one starting with the stress period data DICTIONARY
# We still use the list of lists but now it is stored inside a dictionary 
# with the key representing the stress period number

wel_period_dict = {} # start an empty dictionary
wel1 = [(0,9,9), "my_rates","Mywell"] # changed the assigned rate to a string that will be used by the time series package adding in a boundary name
wel_period_list = [wel1] # remember that the stress_period data is a list of lists
wel_period_dict[1] = wel_period_list # note the dictioanry key is 1, this is stress period two (zero base)

wel = flopy.mf6.ModflowGwfwel(gwf, 
                              boundnames=True, 
                              stress_period_data=wel_period_dict) # Note we changed this to the dictionary 
# added the bound and built the object but didn't setup an observation package
# Lets do it now using an initialization method

# setup the observation dictionary the same as before
wel_obs = {
    "wel_flux.csv": [
        ("Well rate", "WEL", "Mywell")]
}
wel.obs.initialize(continuous=wel_obs)
# check the model object to make sure we haven't doubled up on boundaries
gwf.get_package_list()

In [ ]:
# here I am going to use a sinusoidal function to generate a variable time series

times = np.linspace(1,366,12) # create my sampling periods approximatley monthly over a 366 day period

rates = 10.0 * np.cos(2*np.pi*1/366*times)+10.0 
# 10.0 represents my amplitude, 2*pi means I want a full wave, 
# 1/366 is my frequency so the wave only repeats once every 366 time units
# times is passed in to say "give me the values at each of these sampling times"
# the final 10 is an offset so I don't get the positive to negative swing
plt.plot(times,rates)

In [ ]:
# we need to provide the list of times in model runtime, this means that times are days
# 
ts_data = list(zip(times,rates)) 
# initialize first time series
wel.ts.initialize(
    filename="flow.wel.ts", # here I am telling the constructor what filename to use
    timeseries=ts_data, # This will become the time and value columns
    sfacrecord=-1.0, # this is a scaling factor. Why am I using negative 1.0 here?
    time_series_namerecord='my_rates', # this is the name record that the well package has assigned instead of a rate
    interpolation_methodrecord="linear", # there are different interpolation approaches. Read about them in the MF6io.pdf doc
)

In [ ]:
# write all the files
wel.write()
wel.obs.write()
wel.ts.write()

In [ ]:
# Lets look at the well file
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,'flow.wel'))]

In [ ]:
# Lets look at the well time-series file
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,'flow.wel.ts'))]

In [ ]:
# check the model object package list
gwf.get_package_list()

In [ ]:
# lets run our model again
ws3_mod1_trans(sim,gwf)

Hmmm, looks like the model failed because our time series does not extend beyond the end of the simulation. We'll have to try again.

In [ ]:
# remove the existing wel package from our simulation
gwf.remove_package("WEL")
# Now we build a new one starting with the stress period data DICTIONARY
# We still use the list of lists but now it is stored inside a dictionary 
# with the key representing the stress period number

wel_period_dict = {} # start an empty dictionary
wel1 = [(0,9,9), "my_rates","Mywell"] # changed the assigned rate to a string that will be used by the time series package adding in a boundary name
wel_period_list = [wel1] # remember that the stress_period data is a list of lists
wel_period_dict[1] = wel_period_list # note the dictioanry key is 1, this is stress period two (zero base)

wel = flopy.mf6.ModflowGwfwel(gwf, 
                              boundnames=True, 
                              stress_period_data=wel_period_dict) # Note we changed this to the dictionary 
# added the bound and built the object but didn't setup an observation package
# Lets do it now using an initialization method

# setup the observation dictionary the same as before
wel_obs = {
    "wel_flux.csv": [
        ("Well rate", "WEL", "Mywell")]
}
wel.obs.initialize(continuous=wel_obs)

times = np.linspace(1,730,24) # create my sampling periods approximatley monthly over a two years.
rates = 10.0 * np.cos(2*np.pi*1/365*times)+10.0
# we need to provide the list of times in model runtime, this means that times are days
# 
ts_data = list(zip(times,rates)) 
# initialize first time series
wel.ts.initialize(
    filename="flow.wel.ts", # here I am telling the constructor what filename to use
    timeseries=ts_data, # This will become the time and value columns
    sfacrecord=-1.0, # this is a scaling factor. Why am I using negative 1.0 here?
    time_series_namerecord='my_rates', # this is the name record that the well package has assigned instead of a rate
    interpolation_methodrecord="linear", # there are different interpolation approaches. Read about them in the MF6io.pdf doc
)

In [ ]:
# check the package list
gwf.get_package_list()

In [ ]:
# lets run our model again
ws3_mod1_trans(sim,gwf)

# Lets plot up some information from our observation files

In [ ]:
# Plotting up your observation flux files
# We will use pandas to load the observation csv files
chd_df = pd.read_csv(os.path.join(model_f,'chd_flux.csv'))
wel_df = pd.read_csv(os.path.join(model_f,'wel_flux.csv'))

# make the time column a datetime index based on model start date
start = pd.to_datetime("2023-12-31T00:00:00") # where did this string come from?
time_series_index = start + pd.to_timedelta(chd_df['time'],unit='days') # this is going to give me a time series index

# Change the index
chd_df.index = time_series_index
wel_df.index = time_series_index

# drop the time column because we don't want to plot this
chd_df.drop(columns='time', inplace=True)
wel_df.drop(columns='time', inplace=True)

# the well has a no-data value for the steady-state stress period (MF6 no-data = 3.0E30)
# We need to convert this to 0.0 or it will cause problems with plotting
wel_df.replace(to_replace=3.0E+30, value=np.NaN, inplace=True)

In [ ]:
# Easy way to plot with a pandas dataframe if you want to look at your figures
ax = chd_df.plot() # will plot all columns in the dataframe as time series
ax2=ax.twinx() # add a second y axis
wel_df.plot(ax=ax2,linestyle='--',color='red') # change some of the plot styles
ax.set_ylabel("Flux $m^{3}d^{-1}$") # Have you seen the $$ used to format math text before?
ax2.set_ylabel('Extraction Rate $m^{3}d^{-1}$"')

# Multipliers
Most boundary packages feature a few special types of auxiliary variables. One is a multiplier. The other is actually reserved for transport simulations. Here we will demonstrate the use of an auxiliary variable as a multiplier. The nice thing about using the auxiliary variables is that they can be assigned a time series too so instead of a specified value as shown in the example we can also change the entries from numbers to time series names and add those time series to the existing time series file or create completely separate time series files for the different auxiliary variables. The MF6io.pdf document highlights which variables in each package may be assigned a time series.


In [ ]:
# remove the existing wel package from our simulation
gwf.remove_package("WEL")
# Now we build a new one starting with the stress period data DICTIONARY
# We still use the list of lists but now it is stored inside a dictionary 
# with the key representing the stress period number

wel_period_dict = {} # start an empty dictionary
wel1 = [(0,9,9), "my_rates",1.5,100.0,"Mywell"] # added mymult=1.5 and concentration=100
wel_period_list = [wel1] # remember that the stress_period data is a list of lists
wel_period_dict[1] = wel_period_list # note the dictioanry key is 1, this is stress period two (zero base)

wel = flopy.mf6.ModflowGwfwel(gwf, 
                              boundnames=True,
                              save_flows=True, # telling it to save flows to the budget file
                              auxiliary=['mymult','concentration'], # here I'm assigning 2 auxiliary variables and naming them
                              auxmultname='mymult', # here I am saying that the first auxiliary variable called mymult is a multiplier
                              stress_period_data=wel_period_dict) # Note we changed this to the dictionary 
# added the bound and built the object but didn't setup an observation package
# Lets do it now using an initialization method

# setup the observation dictionary the same as before
wel_obs = {
    "wel_flux.csv": [
        ("Well rate", "WEL", "Mywell")]
}
wel.obs.initialize(continuous=wel_obs)

times = np.linspace(1,730,24) # create my sampling periods approximatley monthly over a two years.
rates = 10.0 * np.cos(2*np.pi*1/365*times)+10.0
# we need to provide the list of times in model runtime, this means that times are days
# 
ts_data = list(zip(times,rates)) 
# initialize first time series
wel.ts.initialize(
    filename="flow.wel.ts", # here I am telling the constructor what filename to use
    timeseries=ts_data, # This will become the time and value columns
    sfacrecord=-1.0, # this is a scaling factor. Why am I using negative 1.0 here?
    time_series_namerecord='my_rates', # this is the name record that the well package has assigned instead of a rate
    interpolation_methodrecord="linear", # there are different interpolation approaches. Read about them in the MF6io.pdf doc
)

In [ ]:
# check the package list
gwf.get_package_list()

In [ ]:
# lets run our model again
ws3_mod1_trans(sim,gwf)

# What did the multiplier do?

In [ ]:
# Plotting up your observation flux files
# We will use pandas to load the observation csv files
chd_df = pd.read_csv(os.path.join(model_f,'chd_flux.csv'))
wel_df = pd.read_csv(os.path.join(model_f,'wel_flux.csv'))

# make the time column a datetime index based on model start date
start = pd.to_datetime("2023-12-31T00:00:00") # where did this string come from?
time_series_index = start + pd.to_timedelta(chd_df['time'],unit='days') # this is going to give me a time series index

# Change the index
chd_df.index = time_series_index
wel_df.index = time_series_index

# drop the time column because we don't want to plot this
chd_df.drop(columns='time', inplace=True)
wel_df.drop(columns='time', inplace=True)

# the well has a no-data value for the steady-state stress period (MF6 no-data = 3.0E30)
# We need to convert this to 0.0 or it will cause problems with plotting
wel_df.replace(to_replace=3.0E+30, value=np.NaN, inplace=True)

# Easy way to plot with a pandas dataframe if you want to look at your figures
ax = chd_df.plot() # will plot all columns in the dataframe as time series
ax2=ax.twinx() # add a second y axis
wel_df.plot(ax=ax2,linestyle='--',color='red') # change some of the plot styles
ax.set_ylabel("Flux $m^{3}d^{-1}$") # Have you seen the $$ used to format math text before?
ax2.set_ylabel('Extraction Rate $m^{3}d^{-1}$"')

# What do these methods cover?
Using the same approach demonstrated in this workshop you should be able to create all of the following boundary conditions:
1. River 
2. General Head Boundary
3. Drain
4. Horizontal Flow Barrier
5. Well
6. Constant head
7. Recharge (list based)
8. Evapotranspiration (list based)

The only difference is the information that needs to be supplied in the stress period data dictionary. For example with the GHB package you would be specifying: 

a model cell, an hydraulic head, a conductance, auxiliary variables, boundary name

There isn't much more to it than that. Selecting your cells can be problematic but we can leverage the intersection objects for that.

# Selcting Cells with grid intersection

In [ ]:
# Selecting cells using Flopy intersection methods
from shapely.geometry import (
    LineString,
    Point,
    Polygon,
)
from flopy.utils import GridIntersect

In [ ]:
# examples of shapefile geometries obtained from shapley or geopandas
# you can create these shapefiles in a GIS if you have a projected grid
# if there is sufficient time then we will demonstarte this too.
poly = Polygon(shell=[(3, 3),(15.6, 7.7),(14.5, 12.9),(4.7, 10.2)])
line = LineString(coordinates=[(5,4),(19,18)])
point = Point([17.6,5.7])

In [ ]:
# create an intersect object
ix = GridIntersect(mg, method="structured")

In [ ]:
#perform all the intersections
result1 = ix.intersect(poly)
result2 = ix.intersect(line)
result3 = ix.intersect(point)

In [ ]:
# get the cell ids covered by the polygon
result1.cellids

In [ ]:
# Use them to make a recharge boundary condition
# annual rainfall mean = 1220 mm
# we will start with 1% of annual rainfall
# estimate of 0.5% to 2% of annual
rain_rate = 1220/365/1000 # (m/d)
min_rate = 0.005*rain_rate
max_rate = 0.03*rain_rate
# our preferred value which is central to log of min and max
pv =  10**(np.log10(min_rate)+((np.log10(max_rate)-np.log10(min_rate))/2)) 

rch_list = [[(0,x[0],x[1]),pv,'poly_rch'] for x in result1.cellids] # make the list with the cell ids. 
#Why is l,r,c assigned like this (0,x[0],x[1])?
rch_period = {} # start the dictionary
rch_period[0]=rch_list 
rch = flopy.mf6.ModflowGwfrch(
    gwf,
    boundnames=True,
    save_flows=True,
    stress_period_data=rch_period,
)
# setup the observation dictionary the same as before
rch_obs = {
    "rch_flux.csv": [
        ("Recharge", "RCH", "poly_rch")]
}
rch.obs.initialize(continuous=rch_obs)

In [ ]:
# Lets get a modelgrid object and have a look at the grid
mg = gwf.modelgrid

# create a figure instance to save to our plots folder
fig,ax = plt.subplots(figsize=(7,5)) # we are creating a figure object here so that we can dictate size note there are mutiple ways to do this

# Create the mapview object of the modelgrid and tell it to use the axes we created with our figure
pmv = flopy.plot.PlotMapView(modelgrid=mg, ax=ax) # note by not specifying a layer here it will assume layer 1

# say that you want the Mapview object to plot the grid with linewidth = 0.3 and colour the lines black
pmv.plot_grid(ax=ax, lw=0.3, color="black")

# Now I also want to plot the boundary condition so I specify the package object to plot
pmv.plot_bc(package=rch, color='blue')


ax.set_title('RCH Boundary Cells')
ax.set_xlabel('Eastings')
ax.set_ylabel('Northings')
# lets save it to our plots folder
figname = os.path.join(plots_f,'RCH_bounadry_cells_LR.png') # note use of the plots folder path.
# If you want to change the file format then change the extension from .png to pdf or just do both
fig.savefig(figname,dpi=300)
figname = os.path.join(plots_f,'RCH_bounadry_cells_LR.pdf')
fig.savefig(figname,dpi=300) 

# Depending on time.
If there is time available then we will have a look at a projected grid with inersection and setting up complex transient boundary conditions that would take a long time with GUI's but is relativley quick with a script. If no time is available then we will leave it for later in the series.

# Thats all folks!
Next session will be on configuring array style properties and boundary conditions.